In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 
import matplotlib.gridspec as gridspec
from  collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.manifold import TSNE as tsne
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
# Load the training datas.
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

In [4]:
train.head(3)

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 138 entries, date to ts_id
dtypes: float64(135), int64(3)
memory usage: 2.5 GB


In [7]:
train.shape

(2390491, 138)

In [12]:
round(train.date.describe(),0)

count    2390491.0
mean         248.0
std          152.0
min            0.0
25%          104.0
50%          254.0
75%          382.0
max          499.0
Name: date, dtype: float64

In [9]:
train.date.min()

0

In [10]:
train.date.max()

499

In [16]:
round(train.resp.describe(),3)

count    2390491.000
mean           0.000
std            0.027
min           -0.549
25%           -0.007
50%            0.000
75%            0.008
max            0.448
Name: resp, dtype: float64

In [20]:
pd.DataFrame({c:(train[c].isna().sum()/len(train))*100 for c in train.columns},index=["% missing values"])

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
% missing values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.922161,0.009329,0.009329,0.672791,0.672791,0.370342,0.370342,0.08036,0.08036,0.0


In [24]:
{c:(train[c].isna().sum()/len(train))*100 for c in train.columns}

{'date': 0.0,
 'weight': 0.0,
 'resp_1': 0.0,
 'resp_2': 0.0,
 'resp_3': 0.0,
 'resp_4': 0.0,
 'resp': 0.0,
 'feature_0': 0.0,
 'feature_1': 0.0,
 'feature_2': 0.0,
 'feature_3': 0.018740919752469262,
 'feature_4': 0.018740919752469262,
 'feature_5': 0.0,
 'feature_6': 0.0,
 'feature_7': 16.445784568944205,
 'feature_8': 16.445784568944205,
 'feature_9': 0.032963939207468256,
 'feature_10': 0.032963939207468256,
 'feature_11': 3.347220299093366,
 'feature_12': 3.347220299093366,
 'feature_13': 0.6422529932135281,
 'feature_14': 0.6422529932135281,
 'feature_15': 0.2795659971110538,
 'feature_16': 0.2795659971110538,
 'feature_17': 16.546182353332433,
 'feature_18': 16.546182353332433,
 'feature_19': 0.032963939207468256,
 'feature_20': 0.032963939207468256,
 'feature_21': 3.4069988132145235,
 'feature_22': 3.4069988132145235,
 'feature_23': 0.6422529932135281,
 'feature_24': 0.6422529932135281,
 'feature_25': 0.2795659971110538,
 'feature_26': 0.2795659971110538,
 'feature_27': 16.5461

In [25]:
train = train.query('weight > 0').reset_index(drop=True)
print(f'Shape of weight=0 removed train dataset: {train.shape}')


Shape of weight=0 removed train dataset: (1981287, 138)


In [26]:
train = train.query('date > 0').reset_index(drop=True)
print(f'Shape of date removed train dataset: {train.shape}')


Shape of date removed train dataset: (1976717, 138)


In [ ]:
features_w_missing = list(train.columns[train.isna().sum()>0])
features_wo_mssing = list(train.columns[train.isna().sum()==0])
print(f'Number of features with missing value = {len(train.isnull().sum()[train.isna().sum()>0])}')

N_FEATURES = len(features_w_missing)
nan_val = train.isna().sum()[train.isna().sum() > 0].sort_values(ascending=False)
# print(nan_val)

fig, axs = plt.subplots(figsize=(20, 18))
sns.barplot(y = nan_val.index[0:N_FEATURES], x = nan_val.values[0:N_FEATURES], alpha = 0.7)
plt.title(f'Missing values of train dataset (Top {N_FEATURES} features)')
plt.xlabel('Number of Missing values')
plt.show()